**Caso de estudio: generación de texto**

---


Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos 

Para este ejemplo usaremos “*Character level language model*” propuesto por Andrej Karpathy en su artículo "*The Unreasonable Effectiveness of Recurrent Neural Networks*"(y parcialmente basado en su implementado en el tutorial "*Generate text with an RNN*" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.

#Ejemplo 1: generando nombre de mascotas

##P0. Importación de librerías

In [14]:
#import base64
#import requests

import tensorflow as tf
from keras.models import Model
from keras.optimizers import SGD #optimizador del Gradiente Descendente (SGD)
from keras.layers import Input, Dense, SimpleRNN
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from keras import backend as K # representar la entrada y salida al modelo e
import numpy as np
np.random.seed(5)


##P1. Descarga y preprocesado de los datos

In [ ]:
fileDL= tf.keras.utils.get_file('perros_nombres.txt','https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/recurrent_network_RNN/Datasets/Panel_Txt_Files/perros_nombres.txt')
texto = open(fileDL, 'rb').read().decode(encoding='latin-1')
nombres= texto.lower()
print(nombres)

In [ ]:
master = "https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/recurrent_network_RNN/Datasets/Panel_Txt_Files/perros_nombres.txt"
req = requests.get(master)
nombres = req.text
nombres= nombres.lower()
print(nombres)

##P2. pasar el texto a números

In [16]:
alfabeto = list(set(nombres))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (tam_datos, tam_alfabeto))

En total hay 2181 caracteres, y el diccionario tiene un tamaño de 29 caracteres.


In [17]:
# Conversión de caracteres a índices y viceversa
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))}
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))}
print(car_a_ind)
print(ind_a_car)

{'\n': 0, '\r': 1, ',': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
{0: '\n', 1: '\r', 2: ',', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'x', 27: 'y', 28: 'z'}


P3.Construcción del modelo RNN

---
Tendrá dos entradas:


*    (un carácter del set de entrenamiento) que se representará en el formato one-hot como un vector con 27 elementos (que es el tamaño del alfabeto)
*   (el estado oculto en el instante de tiempo anterior) que se representará con un vector de 25 elementos (que será el mismo tamaño de la capa oculta)

Además, el modelo generará dos salidas:

 *    (la predicción, o el carácter generado por el modelo) que, al igual que la entrada, se representará como un vector con 27 elementos en formato one-hot
 *    (el estado oculto en el instante de tiempo actual), un vector también de 25 elementos.



 

In [18]:
#Creación de la Red Recurrente en Keras
n_a = 25    # Número de unidades en la capa oculta (neuronas)
entrada  = Input(shape=(None,tam_alfabeto))
a0 = Input(shape=(n_a,))

celda_recurrente = SimpleRNN(n_a, activation='tanh', return_state = True)
#la celda recurrente, tomará la entrada  y el estado oculto anterior, y generará el nuevo estado oculto usando la función de activación tangente hiperbólica
capa_salida = Dense(tam_alfabeto, activation='softmax') #tendra tantas nueronas como el numero de caracteres
#la capa de salida tendra la función de activación softmax, que tomará la activación generada por la celda recurrente y generará la salida  o predicción
salida = []
hs, _ = celda_recurrente(entrada, initial_state=a0)
salida.append(capa_salida(hs))
modelo = Model([entrada,a0],salida)
#vusualizamos el modelo
modelo.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 29)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)        [(None, 25), (None,  1375        input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 29)           754         simple_rnn_1[0][0]         

In [19]:
#creamos el optimizador (Gradiente Descendente) y lo añadimos al modelo
opt = SGD(lr=0.0005)
modelo.compile(optimizer=opt, loss='categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


##P4. Entrenamiento de la Red Recurrente RNN

In [20]:
## EJEMPLOS DE ENTRENAMIENTO
with open(fileDL) as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

s definimos una función que tome uno a uno cada ejemplo de entrenamiento y que genere tres vectores, que serán las entradas al modelo

In [21]:
# Crear ejemplos de entrenamiento usando un generador
def train_generator():
    while True:
        # Tomar un ejemplo aleatorio
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]

        # Convertir el ejemplo a representación numérica
        X = [None] + [car_a_ind[c] for c in ejemplo]

        # Crear "Y", resultado de desplazar "X" un caracter a la derecha
        Y = X[1:] + [car_a_ind['\n']]

        # Representar "X" y "Y" en formato one-hot
        x = np.zeros((len(X),1,tam_alfabeto))
        onehot = to_categorical(X[1:],tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
        x[1:,:,:] = onehot
        y = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)

        # Activación inicial (matriz de ceros)
        a = np.zeros((len(X), n_a))

        yield [x, a], y


###P4.1 parametros y entrenamiento

In [22]:
BATCH_SIZE = 80		# Número de ejemplos de entrenamiento a usar en cada iteración
NITS = 1000			# Número de iteraciones

for j in range(NITS):
  #historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)
  historia = modelo.fit(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)

  # Imprimir evolución del entrenamiento cada 1000 iteraciones
  if j%1000 == 0:
    print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')


Iteración: 0, Error: 3.396321



##P5 Predicción de la RNN

In [23]:
def generar_nombre(modelo,car_a_num,tam_alfabeto,n_a):
    # Inicializar x y a con ceros
    x = np.zeros((1,1,tam_alfabeto,))
    a = np.zeros((1, n_a))

    # Nombre generado y caracter de fin de linea
    nombre_generado = ''
    fin_linea = '\n'
    car = -1

    # Iterar sobre el modelo y generar predicción hasta tanto no se alcance
    # "fin_linea" o el nombre generado llegue a los 50 caracteres
    contador = 0
    while (car != fin_linea and contador != 50):
          # Generar predicción usando la celda RNN
          a, _ = celda_recurrente(K.constant(x), initial_state=K.constant(a))
          y = capa_salida(a)
          prediccion = K.eval(y)

          # Escoger aleatoriamente un elemento de la predicción (el elemento con
          # con probabilidad más alta tendrá más opciones de ser seleccionado)
          ix = np.random.choice(list(range(tam_alfabeto)),p=prediccion.ravel())

          # Convertir el elemento seleccionado a caracter y añadirlo al nombre generado
          car = ind_a_car[ix]
          nombre_generado += car

          # Crear x_(t+1) = y_t, y a_t = a_(t-1)
          x = to_categorical(ix,tam_alfabeto).reshape(1,1,tam_alfabeto)
          a = K.eval(a)

          # Actualizar contador y continuar
          contador += 1

          # Agregar fin de línea al nombre generado en caso de tener más de 50 caracteres
          if (contador == 50):
            nombre_generado += '\n'

    print(nombre_generado)

In [ ]:
# Generar 10 ejemplos de nombres generados por el modelo ya entrenado
for i in range(10):
    generar_nombre(modelo,car_a_ind,tam_alfabeto,n_a)



---



---



#EJEMPLO 2: leyendo un cuento

##P0. importando librerias

In [27]:
import numpy
import sys
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

##P1. Descarga y preprocesado de los datos

In [ ]:
fileDL= tf.keras.utils.get_file('cuentos_infantiles.txt','https://raw.githubusercontent.com/luisFernandoCastellanosG/Machine_learning/master/DeepLearning/recurrent_network_RNN/Datasets/Panel_Txt_Files/varios_cuentos_infantiles.txt')
texto = open(fileDL, 'rb').read().decode(encoding='latin-1')
raw_text = texto.lower()
print(raw_text)

##P2. pasar el texto a números

In [32]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)
print(char_to_int)

['\n', '\r', ' ', '!', '"', ',', '-', '.', '0', '1', '2', '7', '8', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x85', '\x93', '\x94', '\x96', '¡', '¿', 'á', 'é', 'í', 'ñ', 'ó', 'ú']
{'\n': 0, '\r': 1, ' ': 2, '!': 3, '"': 4, ',': 5, '-': 6, '.': 7, '0': 8, '1': 9, '2': 10, '7': 11, '8': 12, ':': 13, ';': 14, '?': 15, 'a': 16, 'b': 17, 'c': 18, 'd': 19, 'e': 20, 'f': 21, 'g': 22, 'h': 23, 'i': 24, 'j': 25, 'k': 26, 'l': 27, 'm': 28, 'n': 29, 'o': 30, 'p': 31, 'q': 32, 'r': 33, 's': 34, 't': 35, 'u': 36, 'v': 37, 'w': 38, 'x': 39, 'y': 40, 'z': 41, '\x85': 42, '\x93': 43, '\x94': 44, '\x96': 45, '¡': 46, '¿': 47, 'á': 48, 'é': 49, 'í': 50, 'ñ': 51, 'ó': 52, 'ú': 53}


In [36]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))

En total hay 55827 caracteres, y el diccionario tiene un tamaño de 54 caracteres.


###P2.1 Dividimos el texto en secuencias:
---
Dividimos el texto en estas secuencias, convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente



In [34]:
# preparar el conjunto de datos de los pares de entrada a salida codificados como enteros
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Se generaron {} secuencias del 100 caracteres desde texto". format(n_patterns))

Se generaron 55727 secuencias del 100 caracteres desde texto


##P3. preparar nuestros datos de entrenamiento

---


1.   Primero debemos transformar la lista de secuencias de entrada en la forma [muestras, pasos de tiempo, características] esperada por una red LSTM.
2.   Luego debemos cambiar la escala de los números enteros al rango de 0 a 1 para que los patrones sean más fáciles de aprender mediante la red LSTM que utiliza la función de activación sigmoidea de forma predeterminada.
3.   por ultimo necesitamos convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot. Esto es para que podamos configurar la red para predecir la probabilidad de cada uno de los 54 caracteres diferentes en el vocabulario (una representación más fácil)







In [37]:
#transformar la lista X de secuencias de entrada en la forma [muestras, pasos de tiempo, características]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalizar (cambiar la escala de los números enteros al rango de 0 a 1 )
X = X / float(n_vocab)
# convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot.
y = np_utils.to_categorical(dataY)

##P4.Construcción del modelo RNN

---
definimos nuestro modelo LSTM: 
Aquí definimos una única capa LSTM oculta con 256 unidades de memoria. La red usa deserción con una probabilidad de 20. La capa de salida es una capa densa que usa la función de activación softmax para generar una predicción de probabilidad para cada uno de los 54 caracteres entre 0 y 1.


In [38]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
#utilizamos el algoritmo de optimización de ADAM para la velocidad
model.compile(loss='categorical_crossentropy', optimizer='adam')

###P4.1 Creando chekpoints

---

La red es lenta de entrenar (alrededor de 300 segundos por época) teniendo activa la GPU, ASí que crearemos CHECKPOINTS (puntos de control) para registrar todos los pesos de la red para archivar cada vez que se observe una mejora en la pérdida al final de la época. Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección

In [39]:
# definimos  los checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

###P4.2 entrenando

In [ ]:
model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list)

##P5.Generando texto con una red LSTM


---
Vamos a cargar el ultimo CHECKPOINT de entrenamiento y con el haremos MAGIA!!!


In [43]:
filename = "/content/weights-improvement-1000-0.3400.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

###P5.1 mapeo inverso (números a letras)
creamos un mapeo inverso que podamos usar para convertir los números enteros nuevamente en caracteres para que podamos entender las predicciones

In [44]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

###P5.2 hacer predicciones
La forma más sencilla de utilizar el modelo Keras LSTM para hacer predicciones es comenzar primero con una secuencia semilla como entrada, generar el siguiente carácter y luego actualizar la secuencia semilla para agregar el carácter generado al final y recortar el primer carácter. Este proceso se repite mientras queramos predecir nuevos caracteres (por ejemplo, una secuencia de 1000 caracteres de longitud).

In [46]:
# elige una semilla al azar
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
 
 p i n o c h o   y   g r i l l o   d e c i d i e r o n   i r   a   b u s c a r l e ,   p e r o   s e   c r u z a r o n   c o n   u n   g r u p o   d e   n i ñ o"
s:
- ¿dónde vais?- preguntó pinocho.


sesii dlcpó mesor er farelterta ale nevor de lu abpilon de la casa de én ceríano ee parno.

- espera  ses rod dono sa sistó ca ellara, es aloa el lasa qu qedin
Done.


##P6.Mejorando la red (una LSTM más grande)

In [47]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
#pero agregaremos una segunda capa.
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
#cambiamos el nombre de archivo de los pesos con puntos de control para que 
#podamos distinguir entre los pesos de esta red 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

###P6.1 mejoramos el entrenamiento

---
aumentamos las epoch y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender.


In [48]:
#los tiempos de entrenamiento aumentaran al doble que en la versión anterior
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
871/871 [==============================] - 21s 22ms/step - loss: 3.0557

Epoch 00001: loss did not improve from 1.34735
Epoch 2/50
871/871 [==============================] - 19s 22ms/step - loss: 2.7523

Epoch 00002: loss did not improve from 1.34735
Epoch 3/50
871/871 [==============================] - 19s 22ms/step - loss: 2.6226

Epoch 00003: loss did not improve from 1.34735
Epoch 4/50
871/871 [==============================] - 19s 22ms/step - loss: 2.5282

Epoch 00004: loss did not improve from 1.34735
Epoch 5/50
871/871 [==============================] - 19s 22ms/step - loss: 2.4478

Epoch 00005: loss did not improve from 1.34735
Epoch 6/50
871/871 [==============================] - 19s 22ms/step - loss: 2.3582

Epoch 00006: loss did not improve from 1.34735
Epoch 7/50
871/871 [==============================] - 19s 22ms/step - loss: 2.2554

Epoch 00007: loss did not improve from 1.34735
Epoch 8/50
871/871 [==============================] - 19s 22ms/step - loss: 2.1692


###P6.2 haciendo predicciones

In [ ]:
# elige una semilla al azar
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

##P7. exportar modelo RNN

In [ ]:
!pip install h5py

In [ ]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentos.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modeloRNN_cuentosPesos.hdf5")
model.save('modelRNN_cuentos_v_h5.h5')
print("modelo salvado en disco")

modelo salvado en disco


###P7.1 cargando un modelo

In [ ]:
# Recrea exactamente el mismo modelo solo desde el archivo
new_model = keras.models.load_model('/content/modelRNN_cuentos_v_h5.h5')

In [ ]:
chars = sorted(list(set("comiendo una manzana")))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))
pattern = dataX[5]
print(pattern)

En total hay 55827 caracteres, y el diccionario tiene un tamaño de 11 caracteres.
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 18, 16, 31, 20, 33, 36, 18, 24, 35, 16, 2, 33, 30, 25, 16, 1, 0, 23, 16, 17, 50, 16, 2, 36, 29, 16, 2, 37, 20, 41, 2, 36, 29, 16, 2, 19, 36, 27, 18, 20, 2, 29, 24, 51, 16, 2, 32, 36, 20, 2, 32, 36, 20, 33, 50, 16, 2, 28, 36, 18, 23, 30, 2, 16, 2, 34, 36, 2, 28, 16, 19, 33, 20, 2, 40, 2, 16, 2, 34, 36, 2, 16, 17, 36]


In [ ]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = new_model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"n t e m p l a r   l a s   f l o r e s ,   q u e   c o m e n z a b a n   a   l l e n a r l o   t o d o .   a l l í   v i o   e n   e l   e s t a n q u e   d o s   d e   a q u e l l o s   p á j a r o s"
 sue a la belle y con la boca abierta, lus amlgos de la casa de los piratas, el pey y el gato com su camino de la casa de su harfio, el príncipe juan, pirocho y la reñora darling se quedó hacia ella.
y se precó ticme que el cartillo y se puedó hacia la casa de su harta que se encontraban en el estuvendo conmigo y sodos que el carciici y se puedó hacia ella.
el patito se sintió delta en palacio, pero el hombre de jengibre corrió más rápido. en la cocina de la casa de los piratas, el peys de noche y la reñora darling se quedó hacia la casa de los piratas,
una teca a tu padre la boca abierta dl la casa y algo suppirar y al canbio, estaba muy bueno y poder vup hijas de podenas del príncipe pue al verla. se acuirtió cnn un cama y deseruer a la princesa y lls piratas a la princesa y l



---



---



#Ejemplo 3: generar texto de cuentos, usando Keras

##P0. importar librerias

In [50]:
import tensorflow as tf
import numpy as np
import os
import time
import sys

##P0. Descarga y preprocesado de los datos

In [51]:
fileDL= tf.keras.utils.get_file('LasMinasDelReySalomon.txt','https://raw.githubusercontent.com/carlos-paezf/Deep_Learning/master/Segundo_Corte/PDF_to_TXT/LasMinasDelReySalomon.txt')
texto = open(fileDL, 'rb').read().decode(encoding='latin-1')

516096/515495 [==============================] - 0s 0us/step


##P1. entendiendo el texto

In [52]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:515495 caracteres
el texto esta compuesto de estos :93 caracteres
['\n', ' ', '!', '"', '&', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x81', '\x9c', '¡', '©', '\xad', '±', '³', 'º', '¼', '¿', 'Â', 'Ã', 'â']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [53]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: ' \n \n \n \n \n \n \n \n  \n \nHenry R. Haggard \n \n \n \n \n \n \nLas minas del Rey SalomÃ³n \n   \n  \n  \n   \n   \n   '
array([ 1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,
        1,  0,  1,  0, 31, 57, 66, 70, 77,  1, 41,  9,  1, 31, 53, 59, 59,
       53, 70, 56,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,
       35, 53, 71,  1, 65, 61, 66, 53, 71,  1, 56, 57, 64,  1, 41, 57, 77,
        1, 42, 53, 64, 67, 65, 91, 86, 66,  1,  0,  1,  1,  1,  0,  1,  1,
        0,  1,  1,  0,  1,  1,  1,  0,  1,  1,  1,  0,  1,  1,  1])


##P3. preparar los datos para ser usados en la RNN

In [54]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

' \n \n \n \n \n \n \n \n  \n \nHenry R. Haggard \n \n \n \n \n \n \nLas minas del Rey SalomÃ³n \n   \n  \n  \n   \n   \n    IntroducciÃ³n \n   \n   \n    Ahora que este libro es'
'tÃ¡ impreso y a punto de salir al mundo, ejerce sobre mÃ\xad un \nenorme peso la conciencia de sus defectos, tanto de estilo como de contenido. En lo \nrefe'
'rente a este Ãºltimo, sÃ³lo puedo decir que no pretende ser una relaciÃ³n exhaustiva de \ntodo lo que vimos e hicimos. Hay muchas cosas concernientes a '
'nuestro viaje a \nKukuanalandia en las que me hubiese gustado explayarme y a las que, de hecho, apenas \naludo. Entre ellas se encuentran las curiosas le'
'yendas que recogÃ\xad sobre las armaduras \nque nos salvaron de la muerte en la gran batalla de Loo, y tambiÃ©n sobre los Silenciosos \no colosos de la entr'
'ada de la cueva de estalactitas. Por otra parte, si me hubiera dejado \nllevar por mis inclinaciones, me habrÃ\xada gustado ahondar en las diferencias, alg'
'unas de \nlas cuales me resultan

###P3.1 separar los datos en agrupamientos (batches)

In [55]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  ' \n \n \n \n \n \n \n \n  \n \nHenry R. Haggard \n \n \n \n \n \n \nLas minas del Rey SalomÃ³n \n   \n  \n  \n   \n   \n    IntroducciÃ³n \n   \n   \n    Ahora que este libro e'
Target data:  '\n \n \n \n \n \n \n \n  \n \nHenry R. Haggard \n \n \n \n \n \n \nLas minas del Rey SalomÃ³n \n   \n  \n  \n   \n   \n    IntroducciÃ³n \n   \n   \n    Ahora que este libro es'


In [56]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [57]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=build_model(vocab_size=vocab_size,
                  embedding_dim=embedding_dim,
                  rnn_units=rnn_units,
                  batch_size=BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=
#    -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23808     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 93)            95325     
Total params: 5,366,109
Trainable params: 5,366,109
Non-trainable params: 0
_________________________________________________________________


##P4. Entrenando la RNN

In [58]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
#con los argumentos por defecto del optimizador Adam.  
model.compile(optimizer='adam',loss=loss)

###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.


In [59]:
checkpoint_dir='/content/trainning_checkpointsCarlos'
checkpoint_prefix= os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

###P4.2 entrenando

In [60]:
EPOCHS=50
history=model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
53/53 [==============================] - 6s 79ms/step - loss: 3.0701
Epoch 2/50
53/53 [==============================] - 5s 79ms/step - loss: 2.4758
Epoch 3/50
53/53 [==============================] - 5s 78ms/step - loss: 2.1997
Epoch 4/50
53/53 [==============================] - 5s 79ms/step - loss: 2.0662
Epoch 5/50
53/53 [==============================] - 5s 79ms/step - loss: 1.9558
Epoch 6/50
53/53 [==============================] - 5s 79ms/step - loss: 1.8547
Epoch 7/50
53/53 [==============================] - 5s 79ms/step - loss: 1.7624
Epoch 8/50
53/53 [==============================] - 5s 79ms/step - loss: 1.6818
Epoch 9/50
53/53 [==============================] - 5s 79ms/step - loss: 1.6079
Epoch 10/50
53/53 [==============================] - 5s 80ms/step - loss: 1.5441
Epoch 11/50
53/53 [==============================] - 5s 80ms/step - loss: 1.4855
Epoch 12/50
53/53 [==============================] - 5s 80ms/step - loss: 1.4368
Epoch 13/50
53/53 [==================

##P5. Generando texto nuevo usando la RNN

In [61]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [62]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.5  
  #entre más alta la temperatura más creatividad al modelo, pero tambien
  #más errores ortograficos.
  model.reset_states()
  #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


###P5.1 generando texto 

In [66]:
print(generate_text(model, start_string=u"estirpe britanica"))

estirpe britanicador 
en la atroumizado autor inversabe el foso de la maÃ±ana, 
que el regimiento terrible no me gusta falla no podÃ­amos distinguir las cazadoras 
de todos los hombres blancos a la mandÃ­bula con un hombre alto, de mayer, es de su parte, si es que 
llegasticio estÃ¡bamos lispuas por su tipo con un suspiro de alivio, y 
no podÃ­amos seguir por allÃ­, a menos que debÃ­amos peser la amplia avenida cuya 
con una cabaÃ±a al que comercial atrÃ¡s, fero no podÃ­amos 
recibir la larga y aplanada su filtr


##P6.exportando modelo

In [ ]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentosV2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modelRNN_cuentosV2_pesos.hdf5")
model.save('modelRNN_cuentosV2.h5')
print("modelo salvado en disco")

modelo salvado en disco
